In [2]:
from application.src.models.faster_rcnn import load_model, RCNNModel
from application.src.config import TEST_DIR
from torchvision.transforms import transforms
from application.src.models.custom_dataset import test_dataset
from PIL import Image
import os

In [3]:
model = load_model()
transform = transforms.Compose([transforms.ToTensor()])

In [8]:
for subdir, dirs, files in os.walk(TEST_DIR):
    for i, file in enumerate(files):
        if file[-4:] == '.jpg':
            img_path = os.path.join(subdir, file)
            print(f'i = {i}, Image path: {img_path}')
            PIL_image = Image.open(img_path)
            test_image = transform(PIL_image)
            test_image = test_image.view(1, 3, test_image.shape[1], test_image.shape[2])
            _, target = test_dataset[i]
            correct_labels, predicted_labels, scores, boxes = model.verify(test_image, target)
            print(f'Correct labels: {correct_labels}')
            print(f'Predicted labels: {predicted_labels}')
            print(f'Scores: {scores}')
            print(f'Boxes: {boxes}')
            if i > 4:
                print('Loop ends')
                break

i = 0, Image path: C:\Users\Kajsa\PycharmProjects\traffic-sign-recognition\application/data/datasets/LU-data/test\1277104195Image000003_jpg.rf.985b035268987fa9c74999f0db77c6b5.jpg
Correct labels: ['100_SIGN']
Predicted labels: ['80_SIGN', '100_SIGN', '70_SIGN']
Scores: [84, 17, 8]
Boxes: [tensor([167.2503, 211.0037, 177.4527, 220.4435]), tensor([167.6144, 210.5826, 177.6667, 220.4621]), tensor([167.3387, 210.7373, 177.6348, 221.5271])]
i = 2, Image path: C:\Users\Kajsa\PycharmProjects\traffic-sign-recognition\application/data/datasets/LU-data/test\1277104195Image000003_jpg.rf.d6a7728a1c45d3746b093c0ca5917553.jpg
Correct labels: ['100_SIGN']
Predicted labels: ['80_SIGN', '70_SIGN', 'PRIORITY_ROAD', '100_SIGN']
Scores: [82, 25, 14, 11]
Boxes: [tensor([357.7505, 448.9753, 379.6328, 470.6000]), tensor([357.3932, 449.2030, 379.7569, 472.8456]), tensor([367.2266, 472.1650, 379.8308, 483.8214]), tensor([358.1158, 448.8360, 379.1151, 471.0172])]
i = 4, Image path: C:\Users\Kajsa\PycharmProject